<a href="https://colab.research.google.com/github/eemlcommunity/PracticalSessions2023/blob/omardd%2Frl/reinforcement_learning/part2_linear_function_approximation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [EEML 2023] Reinforcement Learning Tutorial - Part 2

## RL with Linear Function Approximation

# Colab Setup

In [ ]:
# Colab setup
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
  # install rlberry library (https://github.com/rlberry-py/rlberry)
  !pip install rlberry==0.5.0 > /dev/null 2>&1

  # install ffmpeg-python for saving videos
  !pip install ffmpeg-python > /dev/null 2>&1

  # packages required to show video
  !pip install pyvirtualdisplay > /dev/null 2>&1
  !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# Check rlberry version
import rlberry
print(rlberry.__version__)

# Create directory for saving videos
!mkdir videos > /dev/null 2>&1

# Initialize display and import function to show videos
import rlberry.colab_utils.display_setup
from rlberry.colab_utils.display_setup import show_video

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from rlberry.envs import GridWorld

In [ ]:
def render_policy(env, policy=None, horizon=50):
  """Visualize a policy in an environment

  Args:
    env: GridWorld
        environment where to run the policy
    policy: np.array
        matrix mapping states to action (Ns).
        If None, runs random policy.
    horizon: int
        maximum number of timesteps in the environment.
  """
  env.enable_rendering()
  state, info = env.reset()                       # get initial state
  for timestep in range(horizon):
      if policy is None:
        action = env.action_space.sample()  # take random actions
      else:
        action = policy[state]
      next_state, reward, terminated, truncated, info = env.step(action)
      state = next_state
      if terminated or truncated:
        break
  # save video and clear buffer
  env.save_video('./videos/gw.mp4', framerate=5)
  env.clear_render_buffer()
  env.disable_rendering()
  # show video
  show_video('./videos/gw.mp4')

# Feature Map

In [ ]:
def get_large_gridworld():
  """Creates an instance of a grid-world MDP with more states."""
  walls = [(ii, 10) for ii in range(15) if (ii != 7 and ii != 8)]
  env = GridWorld(
      nrows=15,
      ncols=15,
      reward_at = {(14, 14):1.0},
      walls=tuple(walls),
      success_probability=0.9,
      terminal_states=((14, 14),)
  )
  return env


class GridWorldFeatureMap:
  """Create features for state-action pairs.

  Creates features based on the factorization of a matrix
  containing similarities between states.

  Parameters
  ----------
  dim: int
    Feature dimension
  sigma: float
    RBF kernel bandwidth
  """
  def __init__(self, env, dim=15, sigma=0.25):
    self.index2coord = env.index2coord
    self.n_states = env.Ns
    self.n_actions = env.Na
    self.dim = dim
    self.sigma = sigma

    n_rows = env.nrows
    n_cols = env.ncols

    # build similarity matrix
    sim_matrix = np.zeros((self.n_states, self.n_states))
    for ii in range(self.n_states):
        row_ii, col_ii = self.index2coord[ii]
        x_ii = row_ii / n_rows
        y_ii = col_ii / n_cols
        for jj in range(self.n_states):
            row_jj, col_jj = self.index2coord[jj]
            x_jj = row_jj / n_rows
            y_jj = col_jj / n_cols
            dist = np.sqrt((x_jj - x_ii) ** 2.0 + (y_jj - y_ii) ** 2.0)
            sim_matrix[ii, jj] = np.exp(-(dist / sigma) ** 2.0)

    # factorize similarity matrix to obtain features
    uu, ss, vh = np.linalg.svd(sim_matrix, hermitian=True)
    self.feats = vh[:dim, :]

  def map(self, observation):
    feat = self.feats[:, observation].copy()
    return feat

In [ ]:
env = get_large_gridworld()
feat_map = GridWorldFeatureMap(env)

# Visualize large gridworld
render_policy(env)

# The features have dimension (feature_dim).
feature_example = feat_map.map(1) # feature representation of s=1
print(feature_example)

# Initial vector theta representing the Q function
theta = np.zeros((feat_map.dim, env.action_space.n))
print(theta.shape)
print(feature_example @ theta) # approximation of Q(s=1, a)

# Data Collection Strategies

In [ ]:
def get_random_policy_dataset(env, n_samples):
  """Get a dataset following a random policy to collect data."""
  states = []
  actions = []
  rewards = []
  next_states = []

  state, _ = env.reset()
  for _ in range(n_samples):
    action = env.action_space.sample()
    next_state, reward, terminated, truncated, info = env.step(action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)
    # update state
    state = next_state
    if terminated or truncated:
      state, _ = env.reset()

  dataset = (states, actions, rewards, next_states)
  return dataset

def get_uniform_dataset(env, n_samples):
  """Get a dataset by uniformly sampling states and actions."""
  states = []
  actions = []
  rewards = []
  next_states = []
  for _ in range(n_samples):
    state = env.observation_space.sample()
    action = env.action_space.sample()
    next_state, reward, terminated, truncated, info = env.sample(state, action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)

  dataset = (states, actions, rewards, next_states)
  return dataset


# Collect two different datasets
num_samples = 1000
env = get_large_gridworld()
dataset_1 = get_random_policy_dataset(env, num_samples)
dataset_2 = get_uniform_dataset(env, num_samples)

# Fitted Q-Iteration

Given a datset $(s_i, a_i, r_i, s_i')$ of (states, actions, rewards, next states), the Fitted Q-Iteration (FQI) algorithm proceeds as follows:


* We start from a $Q$ function $Q_0 \in \mathcal{F}$, where $\mathcal{F}$ is a function space;
* At every iteration $k$, we compute $Q_{k+1}$ as:

$$
Q_{k+1}\in\arg\min_{f\in\mathcal{F}} \frac{1}{2}\sum_{i=1}^N
\left(
  f(s_i, a_i) - y_i^k
\right)^2 + \lambda \Omega(f)
$$
where $y_i^k = r_i + \gamma \max_{a'}Q_k(s_i', a')$, $\Omega(f)$ is a regularization term and $\lambda > 0$ is the regularization coefficient.


Consider FQI with *linear* function approximation. That is, for a given feature map $\phi : S \rightarrow \mathbb{R}^d$, we consider a parametric family of $Q$ functions $Q_\theta(s,a) = \phi(s)^T\theta_a$ for $\theta_a\in\mathbb{R}^d$. Suppose we are applying FQI on a given dataset of $N$ tuples of the form $(s_i, a_i, r_i, s_i')$ and we are at the $k$-th iteration. Let $\theta_k \in\mathbb{R}^{d \times A}$ be our current parameter. Derive the *closed-form* update to find $\theta_{k+1}$, using $\frac{1}{2}\sum_a ||\theta_a||_2^2$ as regularization.


Implement Linear FQI in the function below.

### What you need to implement

The solution of the linear system below:

$$
(M_a + \lambda I) \theta_a = b_a
$$

where:

* $M_a = \sum_{i} \mathbb{1}\{a_i = a\}\phi(s_i)\phi(s_i)^T$ is a (d, d) matrix.
* $b_a = \sum_{i} \mathbb{1}\{a_i = a\}\phi(s_i) y_i$ is a (d, 1) matrix.
* $y_i = r_i + \gamma \max_{a'} \phi(s_i', a)^T \theta_a $
* $\mathbb{1}\{a_i = a\}$ is 1 if $a_i=a$ and 0 otherwise.


Notice that $M_a$ does not change every iteration, whereas $b_a$ does (due to the changing targets $y_i$).


In [ ]:
def linear_fqi(env, feat_map, num_iterations):
  # get a dataset
  dataset = get_uniform_dataset(env, n_samples=10000)

  # parameters
  lambd = 0.1
  gamma = 0.95

  theta = np.zeros((feat_map.dim, env.Na))

  # design matrix M_a
  M = np.zeros((env.Na, feat_map.dim, feat_map.dim))
  for state, action, reward, next_state in zip(*dataset):
    state_feat = feat_map.map(state).reshape(-1, 1)
    M[action] += state_feat @ state_feat.T

  for it in tqdm(range(num_iterations), desc="running linear FQI"):
    # build targets for linear regression

    b = np.zeros((env.Na, feat_map.dim))
    for state, action, reward, next_state in zip(*dataset):
      state_feat = feat_map.map(state)
      next_state_feat = feat_map.map(next_state)
      # ====================================================
      # YOUR IMPLEMENTATION HERE - y_i
      target = 0 # ...
      # ====================================================
      b[action] += state_feat * target
    # update theta
    for aa in range(env.Na):
      # ====================================================
      # YOUR IMPLEMENTATION HERE - solving the linear system
      pass
      # theta[:, aa] = ...
      # ====================================================
  return theta


# environment and feature map
env = get_large_gridworld()
# env = get_env()
feat_map = GridWorldFeatureMap(env, dim=100, sigma=0.2)

# FQI
theta = linear_fqi(env, feat_map, num_iterations=100)

# Compute and run greedy policy
Q_fqi = np.zeros((env.Ns, env.Na))
for ss in range(env.Ns):
  state_feat = feat_map.map(ss)
  Q_fqi[ss, :] = state_feat @ theta

V_fqi = Q_fqi.max(axis=1)
policy = Q_fqi.argmax(axis=1)
render_policy(env, policy, horizon=100)
img = env.get_layout_img(V_fqi)
plt.imshow(img)
plt.show()